In [12]:
import os
import sys
import pickle as pkl
import pandas as pd
import src.data.Preprocessing as pproc
import seaborn as sns

repo_path = 'C:\\Users\\Familia\\Desktop\\GitHub\\ada-2023-project-adacadabra2023' # CHANGE
sys.path.append(repo_path)

# from src.data.Data import load_data

In [2]:
dpath = os.path.join(repo_path, 'Data', 'Unified_ratings.pkl')
with open(dpath, 'rb') as f:
    df = pkl.load(f)
print(df.shape)

(12935301, 19)


In [3]:
df

,beer_name,beer_id,brewery_name,brewery_id,style,date,rating,user_id,text,Procedence,country_user,country_brewery,year,Trimester,z_score,exp_mean,diff_exp_mean,ith_rating,detrend
0,Abita Andygator,3.0,Abita Brewing Company,1.0,Maibock / Helles Bock,2003-10-15 10:00:00,3.55,6547.0,I had this one on tap while in New Orleans. I...,BA,"United States, Illinois","United States, Louisiana",2003,T4,-0.254648,-0.254648,0.000000,1,-0.035239
1,Abita Andygator,3.0,Abita Brewing Company,1.0,Maibock / Helles Bock,2003-12-22 11:00:00,3.85,3940.0,Well carbonated greenish gold. The color is a...,BA,"United States, Massachusetts","United States, Louisiana",2003,T4,0.184664,-0.034992,0.219656,2,0.403733
2,Abita Andygator,3.0,Abita Brewing Company,1.0,Maibock / Helles Bock,2004-04-06 10:00:00,2.90,3179.0,Everything about this beer seemed average or ...,BA,"United States, Louisiana","United States, Louisiana",2004,T2,-1.304828,-0.458271,-0.846557,3,-1.086099
3,Abita Andygator,3.0,Abita Brewing Company,1.0,Maibock / Helles Bock,2004-04-22 10:00:00,2.39,3219.0,This can't be a dopplebock. The bar listed it...,BA,"United States, Connecticut","United States, Louisiana",2004,T2,-2.082561,-0.864343,-1.218218,4,-1.864172
4,Abita Andygator,3.0,Abita Brewing Company,1.0,Maibock / Helles Bock,2004-05-02 10:00:00,3.28,19004.0,I had this on tap at the Abita Brew Pub in Ab...,BA,"United States, Texas","United States, Louisiana",2004,T2,-0.725341,-0.836543,0.111202,5,-0.507292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12941878,Arany Ászok Hűsítő Citrom,534175.0,Dreher Sörgyárak (SABMiller),2241.0,Radler/Shandy,2016-07-11 10:00:00,2.00,28282.0,"Can from Tesco, Budapest. Small head with sho...",RB,Denmark,Hungary,2016,T3,-2.381688,-2.405501,0.023813,12,-2.503855
12941879,Arany Ászok Hűsítő Citrom,534175.0,Dreher Sörgyárak (SABMiller),2241.0,Radler/Shandy,2016-07-11 10:00:00,1.40,121469.0,"Can from Tesco, Budapest. Hazy light yellow w...",RB,Denmark,Hungary,2016,T3,-3.394932,-2.481611,-0.913321,13,-3.587319
12941880,Arany Ászok Hűsítő Citrom,534175.0,Dreher Sörgyárak (SABMiller),2241.0,Radler/Shandy,2016-08-21 10:00:00,1.60,122778.0,Pours a clear yellow golden colour with a sma...,RB,Germany,Hungary,2016,T3,-3.057184,-2.522723,-0.534461,14,-3.319791
12941881,Arany Ászok Hűsítő Citrom,534175.0,Dreher Sörgyárak (SABMiller),2241.0,Radler/Shandy,2017-04-15 10:00:00,1.00,391762.0,"bottle @ pub (budapest) THIS IS NOT A BEER, J...",RB,Hungary,Hungary,2017,T2,-4.053868,-2.624800,-1.429068,15,-4.386695


## DELETE WHEN WE DO FILTERING IN THE PREPROCESSING (JUST IMPORT THE FILTERED DF)

In [4]:
def filter_countries(df, min_styles=3):
    ''' Filters countries that do not have at least min_styles styles per trimester.
    Args:
        df: Unified_ratings dataframe coming from main_preprocessing
        min_styles (int) min umber of styles
    '''
    filtered_df = df[['style','Trimester','country_brewery']]

    sty_country = filtered_df.groupby(['country_brewery', 'Trimester'], as_index=False).nunique()

    select_countries = sty_country.groupby('country_brewery', as_index=False).min()
    keep_countries = select_countries[select_countries['style'] >= min_styles]
    valid_countries = keep_countries.country_brewery.values
    filtered_countries = filtered_df[filtered_df['country_brewery'].isin(valid_countries)]
    return filtered_countries

In [5]:
filter_countries(df,3)

## Don't delete what is below

In [4]:
dpath = os.path.join(repo_path, 'Data', 'Unified_users.pkl')
with open(dpath, 'rb') as f:
    df_users = pkl.load(f)
print(df_users.shape)

(45568, 6)


In [5]:
df_users

,nbr_ratings_rb,user_id,user_name,location,nbr_ratings_ba,total_nbr_ratings
0,1890.0,175852.0,Manslow,Poland,0.0,1890.0
1,89.0,442761.0,MAGICuenca91,Spain,0.0,89.0
2,169.0,288889.0,Sibarh,Poland,0.0,169.0
3,3371.0,250510.0,fombe89,Spain,0.0,3371.0
4,13043.0,122778.0,kevnic2008,Germany,0.0,13043.0
...,...,...,...,...,...,...
45563,0.0,36790.0,alfrantzell,"United States, California",43.0,43.0
45564,0.0,36791.0,chefpeet,"United States, New Jersey",22.0,22.0
45565,0.0,36792.0,Dwarf,NaN,21.0,21.0
45566,0.0,36793.0,Tnudish,NaN,34.0,34.0


## DELETE WHAT IS BELOW

In [6]:
# Expertise
df_users['expertise'] = (df_users['total_nbr_ratings'] >= 3352).astype(int)

equivalent_expertise = dict(zip(df_users['user_id'], df_users['expertise']))
df['expertise']=df['user_id'].map(equivalent_expertise)

In [13]:
beer_dict = {
    'American Pale Ale': 'Pale Ale',
    'American Blonde Ale': 'Pale Ale',
    'Golden Ale/Blond Ale': 'Pale Ale',
    'Bitter': 'Pale Ale',
    'Pale Ale': 'Pale Ale',
    'American Pale Ale (APA)': 'Pale Ale',
    'English Pale Ale': 'Pale Ale',
    'English Bitter': 'Pale Ale',
    'Premium Bitter/ESB': 'Pale Ale',
    'English India Pale Ale (IPA)': 'Pale Ale',
    'Extra Special / Strong Bitter (ESB)': 'Pale Ale',
    'Saison / Farmhouse Ale': 'Pale Ale',
    'English Pale Mild Ale': 'Pale Ale',
    'Saison': 'Pale Ale',
    'Belgian Pale Ale': 'Pale Ale',
    'Belgian Strong Pale Ale': 'Strong Pale Ale',
    'Tripel': 'Strong Pale Ale',
    'Abbey Tripel': 'Strong Pale Ale',
    'American IPA': 'IPA',
    'Black IPA': 'IPA',
    'India Pale Ale (IPA)': 'IPA',
    'American Double / Imperial IPA': 'IPA',
    'Imperial IPA': 'IPA',
    'New England IPA (NEIPA)': 'IPA',
    'Session IPA': 'IPA',
    'Belgian IPA': 'IPA',
    'Abbey Dubbel': 'Brown/Dark Ale',
    'Dubbel': 'Brown/Dark Ale',
    'Belgian Dark Ale': 'Brown/Dark Ale',
    'American Brown Ale': 'Brown/Dark Ale',
    'Brown Ale': 'Brown/Dark Ale',
    'English Dark Mild Ale': 'Brown/Dark Ale',
    'American Black Ale': 'Brown/Dark Ale',
    'English Brown Ale': 'Brown/Dark Ale',
    'Belgian Strong Dark Ale': 'Strong Brown/Dark Ale',
    'Abt/Quadrupel': 'Strong Brown/Dark Ale',
    'Barley Wine': 'Strong Brown/Dark Ale',
    'English Barleywine': 'Strong Brown/Dark Ale',
    'American Barleywine': 'Strong Brown/Dark Ale',
    'Quadrupel (Quad)': 'Strong Brown/Dark Ale',
    'Amber Ale': 'Amber Ale',
    'American Amber Ale': 'Amber Ale',
    'Irish Red Ale': 'Amber Ale',
    'American Amber / Red Ale': 'Amber Ale',
    'Red Ale': 'Amber Ale',
    'Belgian Ale': 'Ale',
    'Irish Ale': 'Ale',
    'Old Ale': 'Ale',
    'Mild Ale': 'Ale',
    'Traditional Ale': 'Ale',
    'Scotch Ale / Wee Heavy': 'Ale',
    'Scotch Ale': 'Ale',
    'Scottish Ale': 'Ale',
    'Belgian Strong Ale': 'Strong Ale',
    'American Strong Ale': 'Strong Ale',
    'English Strong Ale': 'Strong Ale',
    'Winter Warmer': 'Strong Ale',
    'Stout': 'Stout',
    'American Stout': 'Stout',
    'Dry Stout': 'Stout',
    'American Double / Imperial Stout': 'Stout',
    'English Stout': 'Stout',
    'Imperial Stout': 'Stout',
    'Irish Dry Stout': 'Stout',
    'Foreign / Export Stout': 'Stout',
    'Oatmeal Stout': 'Stout',
    'Russian Imperial Stout': 'Stout',
    'Milk / Sweet Stout': 'Stout',
    'Foreign Stout': 'Stout',
    'Sweet Stout': 'Stout',
    'Porter': 'Porter',
    'American Porter': 'Porter',
    'Baltic Porter': 'Porter',
    'Imperial Porter': 'Porter',
    'English Porter': 'Porter',
    'American Dark Wheat Ale': 'Wheat Beer',
    'Berliner Weissbier': 'Wheat Beer',
    'Dunkelweizen': 'Wheat Beer',
    'German Hefeweizen': 'Wheat Beer',
    'Hefeweizen': 'Wheat Beer',
    'Wheat Ale': 'Wheat Beer',
    'Weizen Bock': 'Wheat Beer',
    'Roggenbier': 'Wheat Beer',
    'Weizenbock': 'Wheat Beer',
    'Witbier': 'Wheat Beer',
    'German Kristallweizen': 'Wheat Beer',
    'Berliner Weisse': 'Wheat Beer',
    'Kristalweizen': 'Wheat Beer',
    'Wheatwine': 'Wheat Beer',
    'Dunkelweizen': 'Wheat Beer',
    'American Pale Wheat Ale': 'Wheat Beer',
    'Flanders Oud Bruin': 'Wild/Sour Beer',
    'Flanders Red Ale': 'Wild/Sour Beer',
    'Gose': 'Wild/Sour Beer',
    'Lambic - Fruit': 'Wild/Sour Beer',
    'American Wild Ale': 'Wild/Sour Beer',
    'Lambic - Unblended': 'Wild/Sour Beer',
    'Lambic Style - Faro': 'Wild/Sour Beer',
    'Lambic Style - Fruit': 'Wild/Sour Beer',
    'Lambic Style - Gueuze': 'Wild/Sour Beer',
    'Lambic Style - Unblended': 'Wild/Sour Beer',
    'Sour Red/Brown': 'Wild/Sour Beer',
    'Sour/Wild Ale': 'Wild/Sour Beer',
    'Faro': 'Wild/Sour Beer',
    'Gueuze': 'Wild/Sour Beer',
    'Grodziskie/Gose/Lichtenhainer': 'Wild/Sour Beer',
    'American Adjunct Lager': 'Pilsner & Pale Lager',
    'American Pale Lager': 'Pilsner & Pale Lager',
    'Pilsener': 'Pilsner & Pale Lager',
    'Dortmunder / Export Lager': 'Pilsner & Pale Lager',
    'European Pale Lager': 'Pilsner & Pale Lager',
    'German Pilsener': 'Pilsner & Pale Lager',
    'Pale Lager': 'Pilsner & Pale Lager',
    'Munich Helles Lager': 'Pilsner & Pale Lager',
    'Pilsener (German and Czech)': 'Pilsner & Pale Lager',
    'Czech Pilsener': 'Pilsner & Pale Lager',
    'Czech Pilsner (Světlý)': 'Pilsner & Pale Lager',
    'India Style Lager': 'Pilsner & Pale Lager',
    'Imperial Pils/Strong Pale Lager': 'Pilsner & Pale Lager',
    'Kellerbier / Zwickelbier': 'Pilsner & Pale Lager',
    'Euro Pale Lager': 'Pilsner & Pale Lager',
    'Zwickel/Keller/Landbier': 'Pilsner & Pale Lager',
    'Euro Strong Lager': 'Pilsner & Pale Lager',
    'American Double / Imperial Pilsner': 'Pilsner & Pale Lager',
    'Light Lager': 'Pilsner & Pale Lager',
    'Radler/Shandy': 'Pilsner & Pale Lager',
    'Japanese Rice Lager': 'Pilsner & Pale Lager',
    'Premium Lager': 'Pilsner & Pale Lager',
    'Dortmunder/Helles': 'Pilsner & Pale Lager',
    'Doppelbock': 'Bock',
    'Eisbock': 'Bock',
    'Heller Bock': 'Bock',
    'Maibock / Helles Bock': 'Bock',
    'Bock': 'Bock',
    'Amber Lager/Vienna': 'Dark Lager',
    'Munich Dunkel Lager': 'Dark Lager',
    'Schwarzbier': 'Dark Lager',
    'Vienna Lager': 'Dark Lager',
    'American Amber / Red Lager': 'Dark Lager',
    'Dunkel/Tmavý': 'Dark Lager',
    'Dunkler Bock': 'Dark Lager',
    'Euro Dark Lager': 'Dark Lager',
    'Schwarzbier': 'Dark Lager',
    'Polotmavý': 'Dark Lager',
    'Märzen / Oktoberfest': 'Dark Lager',
    'Oktoberfest/Märzen': 'Dark Lager',
    'Rauchbier': 'Smoked',
    'Smoked': 'Smoked',
    'Smoked Beer': 'Smoked',
    'Cream Ale': 'Hybrid Beer',
    'Kölsch': 'Hybrid Beer',
    'California Common': 'Hybrid Beer',
    'California Common / Steam Beer': 'Hybrid Beer',
    'Braggot': 'Hybrid Beer',
    'Bière de Garde': 'Hybrid Beer',
    'Black & Tan': 'Hybrid Beer',
    'Altbier': 'Hybrid Beer',
    'Rye Beer': 'Hybrid Beer',
    'Chile Beer': 'Herbs/Vegetables',
    'Cider': 'Herbs/Vegetables',
    'Fruit / Vegetable Beer': 'Herbs/Vegetables',
    'Fruit Beer': 'Herbs/Vegetables',
    'Herbed / Spiced Beer': 'Herbs/Vegetables',
    'Mead': 'Herbs/Vegetables',
    'Spice/Herb/Vegetable': 'Herbs/Vegetables',
    'Specialty Grain': 'Herbs/Vegetables',
    'Sahti': 'Herbs/Vegetables',
    'Sahti/Gotlandsdricke/Koduõlu': 'Herbs/Vegetables',
    'Scottish Gruit / Ancient Herbed Ale': 'Herbs/Vegetables',
    'Pumpkin Ale': 'Herbs/Vegetables',
    'Perry': 'Herbs/Vegetables',
    'Happoshu': 'Low Alcohol',
    'Low Alcohol': 'Low Alcohol',
    'Low Alcohol Beer': 'Low Alcohol',
    'Kvass': 'Low Alcohol',
    'American Malt Liquor': 'Cocktails',
    'Bière de Champagne / Bière Brut': 'Cocktails',
    'Malt Liquor': 'Cocktails',
    'Saké - Daiginjo': 'Cocktails',
    'Saké - Futsu-shu': 'Cocktails',
    'Saké - Genshu': 'Cocktails',
    'Saké - Ginjo': 'Cocktails',
    'Saké - Honjozo': 'Cocktails',
    'Saké - Infused': 'Cocktails',
    'Saké - Junmai': 'Cocktails',
    'Saké - Koshu': 'Cocktails',
    'Saké - Namasaké': 'Cocktails',
    'Saké - Nigori': 'Cocktails',
    'Saké - Taru': 'Cocktails',
    'Saké - Tokubetsu': 'Cocktails'
}
df['style'] = df['style'].str.strip() # This CAN GO IN THE PREPROCESSING
df['macro_style'] = df['style'].map(beer_dict) # This CAN GO IN THE PREPROCESSING
df['country_brewery'] = df['country_brewery'].apply(pproc.unify_location) # This CAN GO IN THE PREPROCESSING

## Don't delete what is below

In [14]:
palette_styles = dict(zip(df['macro_style'].unique(), sns.color_palette('tab20', 20)[:20]))

In [16]:
def weighted_average(group):
    # Define weights based on 'expertise'
    weights = {1: 1.5, 0: 1}

    # Calculate the weighted average
    weighted_avg = (group['detrend'] * group['expertise'].map(weights)).sum() / group['expertise'].map(weights).sum()

    return pd.Series({'macro_style': group['macro_style'].iloc[0], 'country_brewery': group['country_brewery'].iloc[0], 'final_score': weighted_avg})

weighted_scores = df.groupby(['country_brewery', 'Trimester', 'macro_style'], as_index=False).apply(weighted_average)
weighted_scores = weighted_scores.sort_values(by=['country_brewery', 'Trimester', 'final_score'],  ascending=[True, True, False])
desired_order = ['country_brewery', 'Trimester', 'macro_style', 'final_score']
weighted_scores = weighted_scores[desired_order]
weighted_scores

,country_brewery,Trimester,macro_style,final_score
0,Albania,T1,Dark Lager,-0.978845
1,Albania,T1,Pilsner & Pale Lager,-2.195745
2,Albania,T2,Dark Lager,-1.285498
3,Albania,T2,Pilsner & Pale Lager,-2.121322
4,Albania,T3,Dark Lager,-1.266197
...,...,...,...,...
4805,Wales,T4,Wheat Beer,-0.294702
4794,Wales,T4,Herbs/Vegetables,-0.300357
4797,Wales,T4,Pale Ale,-0.307982
4793,Wales,T4,Dark Lager,-0.360175


In [43]:
top_3_beerstyles = weighted_scores.groupby(['country_brewery', 'Trimester']).head(3)

styles_combined = top_3_beerstyles.groupby(['country_brewery', 'Trimester'], as_index= False).agg(lambda x: x.tolist())

styles_combined[['1st_Style', '2nd_Style', '3rd_Style']] = pd.DataFrame(styles_combined['macro_style'].tolist(), index=styles_combined.index)
styles_combined[['1st_Score', '2nd_Score', '3rd_Score']] = pd.DataFrame(styles_combined['final_score'].tolist(), index=styles_combined.index)
styles_combined = styles_combined.drop(['macro_style', 'final_score'], axis=1)
styles_combined

,country_brewery,Trimester,1st_Style,2nd_Style,3rd_Style,1st_Score,2nd_Score,3rd_Score
0,Albania,T1,Dark Lager,Pilsner & Pale Lager,None,-0.978845,-2.195745,NaN
1,Albania,T2,Dark Lager,Pilsner & Pale Lager,None,-1.285498,-2.121322,NaN
2,Albania,T3,Dark Lager,Pilsner & Pale Lager,None,-1.266197,-2.152081,NaN
3,Albania,T4,Dark Lager,Pilsner & Pale Lager,None,-1.514507,-2.414001,NaN
4,Andorra,T1,Pilsner & Pale Lager,Amber Ale,Pale Ale,-1.327943,-1.512699,-1.525044
...,...,...,...,...,...,...,...,...
523,Virgin Islands (U.S.),T4,IPA,Amber Ale,Wheat Beer,-0.481893,-0.909342,-0.945725
524,Wales,T1,Amber Ale,IPA,Wild/Sour Beer,0.394771,0.161345,0.100542
525,Wales,T2,Amber Ale,IPA,Strong Brown/Dark Ale,0.306489,0.186952,0.141831
526,Wales,T3,IPA,Amber Ale,Strong Brown/Dark Ale,0.219864,0.154380,0.010825


In [13]:
#styles_combined.to_pickle('training_df.pkl')